Keong Jo Hsi 1001685  
Homework 4 (Coding Part 1)

In [1]:
import os
import torch

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

In [3]:
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import math

import torchvision.models as models
import torchvision.transforms as transforms

%matplotlib notebook
from matplotlib import pyplot as plt

import PIL.Image as Image

In [4]:
def parsefile(f):
    return np.genfromtxt(f, dtype=None, encoding='utf-8')

In [5]:
class dataset_flowers(Dataset):

    def __init__(self, img_dir='/home/ubuntu/data1/flowersstuff/flowers_data/jpg/', transform=None, mode='train'):
        self.img_dir = img_dir
        self.transform = transform

        if mode == 'train':
            dataset = parsefile('/home/ubuntu/data1/flowersstuff/trainfile.txt')
        elif mode == 'val':
            dataset = parsefile('/home/ubuntu/data1/flowersstuff/valfile.txt')
        self.dataset = dataset

    def __getitem__(self, index):

        jpg, label = self.dataset[index]
        imgfile = os.path.join(self.img_dir, jpg)
        img = Image.open(imgfile).convert('RGB')

        if self.transform:
            augmented_img = self.transform(img)
        else:
            trans = transforms.Compose([
                transforms.Resize(224),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
            ])
            augmented_img = trans(img)

        return {'img': augmented_img, 'label': label}

    def __len__(self):
        return len(self.dataset)

#### With Pretraining

In [6]:
device = torch.device('cuda:0')
model = models.resnet18(pretrained=True).to(device)
optimizer = optim.SGD(params=model.parameters(), lr=1e-1)
criterion = nn.CrossEntropyLoss()

In [7]:
datasets = {x: dataset_flowers(mode=x) for x in ['train', 'val']}
batchsize = 12
dl = {x: DataLoader(dataset=datasets[x], batch_size=batchsize, shuffle=True) for x in ['train', 'val']}
dataset_length = {x: len(datasets[x]) for x in ['train', 'val']}
total_iter = {x: math.ceil(dataset_length[x] / batchsize) for x in ['train', 'val']}
num_epochs = 5

In [8]:
since = time.time()
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    
    # each epoch has a training and a validation phase
    for phase in ['train', 'val']:
        iteration = 0
        if phase == 'train':
            model.train() 
        else:
            model.eval()
            
        running_loss = 0.0
        running_corrects = 0
        
        for data_dict in dl[phase]:
            iteration += 1
            data = data_dict['img'].to(device)  # [5,3,244,244], FloatTensor
            label = data_dict['label'].long().to(device) # [5], LongTensor

            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                output = model(data) # [5,1000], FloatTensor
            #     print('output', output)

                pred = (torch.argmax(output, dim=1)) # [5], FloatTensor
            #     print('pred', pred)
            #     print('label', label)

                loss = criterion(output, label)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # minibatch_loss * minibatch_size
            running_loss += loss.item() * label.shape[0]
            running_corrects += sum(torch.eq(pred,label)).item()
            
            print('iter: {}/{}, running loss = {:.4f}'.format(iteration, total_iter[phase], running_loss), end='\r')

        epoch_loss = running_loss / dataset_length[phase]
        epoch_acc = running_corrects / dataset_length[phase]
        
        print('Phase: {} \tLoss: {:.4f} \tAccuracy: {:.4f}'.format(
                phase.capitalize(), epoch_loss, epoch_acc))
        
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            
    print()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
print('Best val acc: {:4f}'.format(best_acc))
model.load_state_dict(best_model_wts)

Epoch 0/4
Phase: Train 	Loss: 4.6645 	Accuracy: 0.0526
Phase: Val 	Loss: 3.8993 	Accuracy: 0.0874

Epoch 1/4
Phase: Train 	Loss: 3.4183 	Accuracy: 0.1667
Phase: Val 	Loss: 3.8539 	Accuracy: 0.2051

Epoch 2/4
Phase: Train 	Loss: 2.5908 	Accuracy: 0.3337
Phase: Val 	Loss: 2.1309 	Accuracy: 0.4461

Epoch 3/4
Phase: Train 	Loss: 1.7284 	Accuracy: 0.5242
Phase: Val 	Loss: 1.5340 	Accuracy: 0.5850

Epoch 4/4
Phase: Train 	Loss: 1.1329 	Accuracy: 0.6838
Phase: Val 	Loss: 0.9622 	Accuracy: 0.7328

Training complete in 10m 34s
Best val acc: 0.732843


#### Pretraining with last layer removed

#### Without Pretraining